# 默认代码

In [ ]:
# 导入形式（必须是双引号）

# %%capture
# %run "viz_Tool.ipynb"

# 最好同时也导入 file_Tool.ipynb 文件，因为下面的绘图方法中会调用到 file_Tool.ipynb 中的数据处理方法

In [ ]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.charts import Scatter3D
from pyecharts.charts import Scatter
from pyecharts.charts import Bar3D
from pyecharts.faker import Faker

In [ ]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

# 折线图
>- 具体语法见 `viz_Use.ipynb` 作图规范

In [ ]:
# X 、Y 可以是 List 、ndarray 、DataFrame 的一列 Series

# ignnore 表示是否将 Y 数据中的 0.0 转化为 None
#         以获得更加平滑的折线图，而不是忽上忽下的尖山图
#         但是两种方法的均值计算不同，需要注意

# Y_name 为 Y 图例名称
# Y_unit 为 Y 数据单位


def draw_Line(X , Y , ignore = False , Y_name = '' , Y_unit = ''):
    
    # 预处理
    X = list(X)
    Y = list(Y)
    
    # X 最好最好是 List [String , String , ...] 且无重复
    X = [str(x) for x in X]
    
    # 是否将 0 或 0.0 转化为 None
    # file_Tool.ipynb 中的数据处理方法
    if ignore:
        Y = trans_Y_for_Viz(Y)
    
    # X 与 Y 长度不一致
    if len(X) < len(Y):
        print('数据 X 有缺失，长度与数据 Y 不一致，可以绘画，但不准确！！！')
    elif len(X) > len(Y):
        print('数据 Y 有缺失，长度与数据 X 不一致，可以绘画，但不准确！！！')
    
    line = (
        Line(
            opts.InitOpts(
                bg_color = '#FFFFFF' ,
                width = '1333px' ,
                height = '600px'
            )
        )
        .add_xaxis(X)
        .add_yaxis(
            Y_name , 
            Y , 
            color = '#FF4500' ,
            symbol_size = [7 , 7] ,
            is_connect_nones = True ,
            markpoint_opts = opts.MarkPointOpts(
                data = [
                    opts.MarkPointItem(type_ = 'min' , name = 'Min') ,          # 标记 min
                    opts.MarkPointItem(type_ = 'max' , name = 'Max')            # 标记 max
                ] ,
                symbol_size = [45 , 45] , 
                label_opts = opts.LabelOpts(
                    formatter = '{b}' , 
                    position = 'inside'
                )
            ) ,
            markline_opts = opts.MarkLineOpts(
                symbol = ['circle' , 'triangle'] ,
                symbol_size = [11 , 11] ,
                label_opts = opts.LabelOpts(
                    font_weight = 'bold' , 
                    font_size = 16 , 
                    color = 'auto' , 
                    formatter = ' {c} 均值线'
                ) ,
                linestyle_opts = opts.LineStyleOpts(
                    width = 1.5 , 
                    type_ = 'dashed'
                ) , 
                data = [
                    opts.MarkLineItem(type_ = 'average' , name = Y_name)       # 标记 mean ，会忽略 None 计算 mean
                ]
            ) , 
            label_opts = opts.LabelOpts(is_show = True , font_weight='bold' , color = 'auto') ,
            linestyle_opts = opts.LineStyleOpts(width = 2.33)
        )
        .set_global_opts(
            datazoom_opts = opts.DataZoomOpts(           # 区域缩放滚动
                range_start = 0 , 
                range_end = 100
            ) ,
            yaxis_opts = opts.AxisOpts(
                axislabel_opts = opts.LabelOpts(
                    formatter = '{value} ' + Y_unit , 
                    font_weight = 'bold' , 
                    font_size = 14 , 
                    color = 'auto')
            ) , 
            xaxis_opts = opts.AxisOpts(
                axislabel_opts = opts.LabelOpts(
                    is_show = True ,
                    formatter = '{value}' , 
                    font_weight = 'bold' , 
                    font_size = 14 , 
                    color = 'auto' , 
                    rotate = 0)
            ) ,
            legend_opts = opts.LegendOpts(pos_top = 15) ,
            toolbox_opts = opts.ToolboxOpts(pos_top = 10 , pos_left = 950) ,
            tooltip_opts=opts.TooltipOpts(
                is_show = True , 
                trigger = 'axis' , 
                axis_pointer_type = 'cross')
        )
    )
    return line

# 散点图
>- 一般用于可视化坐标点 `(x , y)` 或 `(x , y , z)` 的分布情况
>- 所以传入的 `X 、Y 、Z` 数据都必须为数值型
>  - `int` 整数
>  - `float` 浮点数

## `3D` 散点图
>- [`3D` 文档](https://pyecharts.org/#/zh-cn/3d_charts)
>- [视觉映射配置](https://pyecharts.org/#/zh-cn/global_options?id=visualmapopts%ef%bc%9a%e8%a7%86%e8%a7%89%e6%98%a0%e5%b0%84%e9%85%8d%e7%bd%ae%e9%a1%b9)

In [ ]:
# X 、Y 、Z 
#     可以是 List 、ndarray 、DataFrame 的一列 Series
#     数据必须是 int 或 float
#     长度必须都相同

# _name 参数为 X 、Y 、Z 轴的名称
# title 是图例名称

def draw_Scatter3D(X , Y , Z , X_name = 'X' , Y_name = 'Y' , Z_name = 'Z' , title = '图例名称'):
    
    # 预处理
    X = list(X)
    Y = list(Y)
    Z = list(Z)
    # print(len(X))
    # print(len(Y))
    # print(len(Z))
    
    if len(X) != len(Y) or len(X) != len(Z):
        print('数据 X 、Y 与 Z 长度不一致，无法绘制 3D 散点图！！！')
    else:
        
        XYZ = [(X[i] , Y[i] , Z[i]) for i in range(len(X))]
        
        scatter = (
            Scatter3D(
                opts.InitOpts(
                    bg_color = '#FFFFFF' ,
                    width = '1333px' ,
                    height = '600px'
                )
            )
            .add(
                title , 
                XYZ , 
                grid3d_opts = opts.Grid3DOpts(           # 笛卡尔坐标系样式
                    height = 90 ,                        # Z 轴长度
                    width =  175 ,                       # X 轴长度
                    depth = 90 ,                         # Y 轴长度
                    rotate_speed = 10 , 
                    rotate_sensitivity = 3               # 鼠标转动的灵敏度
                ) , 
                xaxis3d_opts = opts.Axis3DOpts(          # X_name 为 X 轴名称
                    type_ = 'value' , 
                    name = X_name , 
                    # min_ = 'dataMin' , 
                    # max_ = 'dataMax' , 
                ) , 
                yaxis3d_opts = opts.Axis3DOpts(          # Y_name 为 Y 轴名称
                    type_ = 'value' , 
                    name = Y_name , 
                    # min_ = 'dataMin' , 
                    # max_ = 'dataMax' , 
                ) , 
                zaxis3d_opts = opts.Axis3DOpts(          # Z_name 为 Z 轴名称
                    type_ = 'value' , 
                    name = Z_name , 
                    # min_ = 'dataMin' , 
                    # max_ = 'dataMax' , 
                ) , 
            )
            .set_global_opts(
                visualmap_opts = opts.VisualMapOpts(     # 按 Z 轴高低区分色彩
                    min_ = min(Z) ,
                    max_ = max(Z) , 
                    range_text = [f'High' , f'Low'] , 
                    range_color = Faker.visual_color ,   # 配色
                    pos_top = 188 , 
                    pos_left = 80 , 
                ) , 
                toolbox_opts = opts.ToolboxOpts(pos_top = 10 , pos_left = 950) ,       # 便捷工具栏
            )
        )
        return scatter

## `3D` 散点图标签
>- 根据每个三维坐标点 `(x , y , z)` 的类别或标签 `lable` 不同，显式不同的颜色

In [ ]:
# X_Y_Z 数据为如下两种结构
# [
#     [x , y , z] , 
#     [x , y , z] , 
#     [x , y , z] , 
#     ... ,
# ]
# [
#     (x , y , z) , 
#     (x , y , z) , 
#     (x , y , z) , 
#     ... ,
# ]

# labels = [int , int , ...] 或 [String , String , ...]
# labelsDict = {
#     标签 : '颜色' , 
#     标签 : '颜色' , 
#     ...
# }

# 常用的一些标签颜色字典，即 Faker.visual_color 配色
LABELS_DICT_Int = {
    0 : '#abd9e9' , 
    1 : '#fee090' , 
    2 : '#fdae61' ,
    3 : '#f46d43' , 
    4 : '#4575b4' , 
}
LABELS_DICT_Str = {
    'A' : '#abd9e9' , 
    'B' : '#fee090' , 
    'C' : '#fdae61' ,
    'D' : '#f46d43' , 
    'E' : '#4575b4' , 
}


def draw_Scatter3D_clf(X_Y_Z , 
                       labels , labelsDict , 
                       X_name = 'X' , Y_name = 'Y' , Z_name = 'Z' , 
                       title = '图例名称'):
    
    # 下面要用到 Z 确定 VisualMapOpts 的范围
    Z = [ xyz[2] for xyz in X_Y_Z ]
    
    scatter3d = (
        Scatter3D(
            opts.InitOpts(
                bg_color = '#FFFFFF' ,
                width = '1333px' ,
                height = '600px'
            )
        )
        .add(
            title , 
            [ {'value':xyz} for xyz in X_Y_Z ] ,     # 将 X_Y_Z 转化为字典
            grid3d_opts = opts.Grid3DOpts(           # 笛卡尔坐标系样式
                height = 90 ,                        # Z 轴长度
                width =  175 ,                       # X 轴长度
                depth = 90 ,                         # Y 轴长度
                rotate_speed = 10 , 
                rotate_sensitivity = 3               # 鼠标转动的灵敏度
            ) , 
            xaxis3d_opts = opts.Axis3DOpts(          # X_name 为 X 轴名称
                type_ = 'value' , 
                name = X_name , 
                # min_ = 'dataMin' , 
                # max_ = 'dataMax'
            ) , 
            yaxis3d_opts = opts.Axis3DOpts(          # Y_name 为 Y 轴名称
                type_ = 'value' , 
                name = Y_name , 
                # min_ = 'dataMin' , 
                # max_ = 'dataMax'
            ) , 
            zaxis3d_opts = opts.Axis3DOpts(          # Z_name 为 Z 轴名称
                type_ = 'value' , 
                name = Z_name , 
                # min_ = 'dataMin' , 
                # max_ = 'dataMax'
            ) , 
        )
        .set_global_opts(
            visualmap_opts = opts.VisualMapOpts(     # 按 Z 轴高低区分色彩
                min_ = min(Z) ,
                max_ = max(Z) , 
                range_text = [f'High' , f'Low'] , 
                range_color = Faker.visual_color ,   # 配色
                pos_top = 188 , 
                pos_left = 80 , 
            ) , 
            toolbox_opts = opts.ToolboxOpts(pos_top = 10 , pos_left = 950) ,       # 便捷工具栏
        )
    )
    
    # 根据 labels 和 labelsDict 设置每个点的颜色
    for i, la in enumerate(labels):
        scatter3d.options['series'][0]['data'][i].update(
            itemStyle = { 
                'color' : labelsDict[la] 
            }
        )
    
    return scatter3d

## `2D` 散点图
>- [文档](https://pyecharts.org/#/zh-cn/rectangular_charts?id=scatter%ef%bc%9a%e6%95%a3%e7%82%b9%e5%9b%be)

In [ ]:
# X 、Y
#      可以是 List 、ndarray 、DataFrame 的一列 Series
#      数据必须是 int 或 float
#      长度必须都相同

# 控制参数：
#     scale 参数在 X 、Y 数据的最大值和最小值基础上，用于微微调节 X 、Y 轴的刻度范围
#     c     是否使用 VisualMapOpts ，默认为 True ，将覆盖 cc 的统一颜色
#           且不知道为啥，使用 VisualMapOpts 后无鼠标移动提示框，即 TooltipOpts 也会失效
#           使用统一颜色 cc 就有鼠标移动提示框
#           但 3d 图即使使用 VisualMapOpts 也会有鼠标移动提示框
#     cc    统一的散点颜色，只有在 c = False 的情况下才有效

def draw_Scatter2D(X , Y , Xscale = 0 , Yscale = 0 , c = True , cc = '#00BFFF' , title = '图例名称'):
    
    # 预处理
    X = list(X)
    Y = list(Y)
    
    if len(X) != len(Y):
        print('数据 X 与 Y 长度不一致，无法绘制 2D 散点图！！！')
    else:
        scatter = (
            Scatter(
                opts.InitOpts(
                    bg_color = '#FFFFFF' ,
                    width = '1333px' ,
                    height = '600px'
                )
            )
            .add_xaxis(X)
            .add_yaxis(
                title ,
                Y , 
                color = cc , 
                symbol_size = 15 ,
                label_opts = opts.LabelOpts(is_show = False) ,     # 不显示每个散点的数据
            )
            .set_global_opts(
                datazoom_opts = opts.DataZoomOpts(                 # 区域缩放滚动
                    range_start = 0 , 
                    range_end = 100
                ) ,
                tooltip_opts = opts.TooltipOpts(          # 鼠标移动提示框
                    is_show = True , 
                    trigger = 'axis' , 
                    axis_pointer_type = 'cross') , 
                xaxis_opts = opts.AxisOpts(               # X 轴刻度样式
                    type_ = 'value' ,                     # 说明 X 轴是数值轴，适用于连续数据
                    min_ = min(X) - Xscale ,              # 刻度范围
                    max_ = max(X) + Xscale ,
                ),
                yaxis_opts = opts.AxisOpts(               # Y 轴刻度样式
                    type_ = 'value' ,                     # 说明 Y 轴是数值轴，适用于连续数据
                    min_ = min(Y) - Yscale ,              # 刻度范围
                    max_ = max(Y) + Yscale , 
                ) , 
                toolbox_opts = opts.ToolboxOpts(pos_top = 10 , pos_left = 950) ,       # 便捷工具栏
            )
        )
        if c:
            scatter.set_global_opts(
                visualmap_opts = opts.VisualMapOpts(      # Y 轴高低区分色彩
                    min_ = min(Y) ,
                    max_ = max(Y) , 
                    range_text = [f'High' , f'Low'] , 
                    range_color = Faker.visual_color ,    # 配色
                    pos_top = 188 , 
                    pos_left = 0 , 
                ) , 
                toolbox_opts = opts.ToolboxOpts(pos_top = 10 , pos_left = 950) ,       # 便捷工具栏
            )
        
        return scatter

## `2D` 散点图叠加
>- 数据聚类分析，详见 `viz_Use.ipynb` 中的测试

In [ ]:
# 上述 draw_Scatter2D() 的精简版本，或者说是适配版，专用于数据聚类分析的绘制
#     仅在下面的 draw_Scatter2D_clf 方法中调用
#     平常绘制散点图，使用上面的 、更加通用的 draw_Scatter2D()

# 与上述 draw_Scatter2D() 的区别：
#     无 c
#     scales 用于直接指定 X 、Y 轴的刻度范围，而不是调节，方便多个类别的散点图 overlap 叠加时统一指定 X 、Y 轴刻度范围
#     scale  同上，用于微微调节

# 注意，每个叠加的散点图的图例 Y_title 必须不同，不然颜色会覆盖


def draw_Scatter2D_special(X , Y , 
                           Xscales = [0 , 0] , Yscales = [0 , 0] , 
                           Xscale = 0 , Yscale = 0 , 
                           cc = '#00BFFF' , 
                           title = '图例名称'):
    
    # 预处理
    X = list(X)
    Y = list(Y)
    
    # X 、Y 轴刻度范围
    if Xscales == [0 , 0]:
        Xscales = [min(X) , max(X)]
    if Yscales == [0 , 0]:
        Yscales = [min(Y) , max(Y)]
    
    if len(X) != len(Y):
        print('数据 X 与 Y 长度不一致，无法绘制 2D 散点图！！！')
    else:
        scatter = (
            Scatter(
                opts.InitOpts(
                    bg_color = '#FFFFFF' ,
                    width = '1333px' ,
                    height = '600px'
                )
            )
            .add_xaxis(X)
            .add_yaxis(
                title ,
                Y , 
                color = cc , 
                symbol_size = 15 ,
                label_opts = opts.LabelOpts(is_show = False) ,
            )
            .set_global_opts(
                datazoom_opts = opts.DataZoomOpts(
                    range_start = 0 , 
                    range_end = 100
                ) ,
                tooltip_opts = opts.TooltipOpts(
                    is_show = True , 
                    trigger = 'axis' , 
                    axis_pointer_type = 'cross') , 
                xaxis_opts = opts.AxisOpts(               # X 轴刻度样式
                    type_ = 'value' ,
                    min_ = Xscales[0] - Xscale ,
                    max_ = Xscales[1] + Xscale ,
                ),
                yaxis_opts = opts.AxisOpts(               # Y 轴刻度样式
                    type_ = 'value' ,
                    min_ = Yscales[0] - Yscale ,
                    max_ = Yscales[1] + Yscale , 
                ) , 
                toolbox_opts = opts.ToolboxOpts(pos_top = 10 , pos_left = 950) ,       # 便捷工具栏
            )
        )
        
        return scatter

In [ ]:
# clf == classfier == 分类绘制

# data 为 DataFrame
#      col1    要作为 X 数据的列名
#      col2    要作为 Y 数据的列名
#      colLab  用于分类的标签列的列名

# colors 为每个类别的颜色列表 List ，需要与标签类别个数相同
# scale 同上，为微微调整

# 常用颜色
COLORS = [
    '#FF0000' , '#1E90FF' , '#54FF9F' ,    # 红 蓝 绿
    '#FFDAB9' , '#ffff00' , '#313695' ,    # 乳 黄 深蓝
    '#FF00FF' , '#FF7F24' , '#8B0000' ,    # 橘 紫 酒红 
    '#abd9e9' ,    # 浅蓝
]


def draw_Scatter2D_clf(data , col1 , col2 , colLab , colors = COLORS , Xscale = 0 , Yscale = 0):
    
    
    # 统计所有的类别标签
    # file_Tool.ipynb 中的数据处理方法
    cnts = val_cnt(data , colLab)
    labels = list(cnts[colLab].values)
    labels.sort()
    
    if len(labels) > len(colors):
        print('提供的颜色列表 colors 的长度 < 标签类别个数，无法绘制！！！')
    else:
        
        # 统一多个散点图叠加时的 X 、Y 轴刻度范围
        minX = min(data[col1])
        maxX = max(data[col1])
        minY = min(data[col2])
        maxY = max(data[col2])
        
        # 每个类别的散点图
        scatters = []
        
        # 遍历颜色
        idx = 0
        
        for la in labels:
            scatters += [
                draw_Scatter2D_special(
                    X = data[data[colLab] == la][col1] , 
                    Y = data[data[colLab] == la][col2] , 
                    Xscales = [minX , maxX] , 
                    Yscales = [minY , maxY] , 
                    Xscale = Xscale , 
                    Yscale = Yscale , 
                    cc = colors[idx] , 
                    title = f'Class-{la}'
                )
            ]
            idx += 1
        
        # 叠加多个散点图
        for i in range(1 , len(scatters)):
            scatters[0].overlap(scatters[i])
            
        return scatters[0]

# `3D` 柱状图
>- 同散点图， `X 、Y 、Z` 数据都必须为数值型（`int` 或 `float`）
>- 但可以传入 `Xticks` 和 `Yticks` ，表示 `X 、Y` 轴的刻度标签

In [ ]:
# X_Y_Z 数据为如下两种结构
# [
#     [x , y , z] , 
#     [x , y , z] , 
#     [x , y , z] , 
#     ... ,
# ]
# [
#     (x , y , z) , 
#     (x , y , z) , 
#     (x , y , z) , 
#     ... ,
# ]


def draw_Bar3D(X_Y_Z , 
               Xticks , Yticks , 
               X_name = 'X' , Y_name = 'Y' , Z_name = 'Z' , title = '图例名称'
              ):
    
    bar3d = (
        Bar3D(
            opts.InitOpts(
                bg_color = '#FFFFFF' ,
                width = '1333px' ,
                height = '600px'
            )
        )
        .add(
            title ,
            X_Y_Z ,
            grid3d_opts = opts.Grid3DOpts(     # 笛卡尔坐标系样式
                height = 90 ,
                width =  175 ,
                depth = 90 ,
                rotate_speed = 10 , 
                rotate_sensitivity = 3
            ) , 
            xaxis3d_opts = opts.Axis3DOpts(    # X 轴刻度，大概因为 type_ = 'category' 所以可以自定义 ticks
                type_ = 'category' ,           # 上述的 3D 散点图也可以
                data = Xticks , 
                name = X_name , 
                min_ = 'dataMin' , 
                max_ = 'dataMax' , 
            ) ,
            yaxis3d_opts = opts.Axis3DOpts(    # Y 轴刻度
                type_ = 'category' , 
                data = Yticks , 
                name = Y_name , 
                min_ = 'dataMin' , 
                max_ = 'dataMax' , 
            ) ,
            zaxis3d_opts = opts.Axis3DOpts(
                type_ = 'value' , 
                name = Z_name , 
                min_ = 'dataMin' , 
                max_ = 'dataMax' , 
            ) , 
        )
        .set_global_opts(
            visualmap_opts = opts.VisualMapOpts(     # 按 Z 轴高低区分色彩
                min_ = min(Z) ,
                max_ = max(Z) , 
                range_text = [f'High' , f'Low'] , 
                range_color = Faker.visual_color ,   # 配色
                pos_top = 188 , 
                pos_left = 80 , 
            ) , 
            toolbox_opts = opts.ToolboxOpts(pos_top = 10 , pos_left = 950) ,       # 便捷工具栏
        )
    )
    return bar3d